In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Pool
import datetime as dt
import gc

In [ ]:
root_path = '../input/optiver-realized-volatility-prediction'
# read book data
book_train = pd.read_parquet(os.path.join(root_path, 'book_train.parquet'))
book_test = pd.read_parquet(os.path.join(root_path, 'book_test.parquet'))
trade_train = pd.read_parquet(os.path.join(root_path, 'trade_train.parquet'))
trade_test = pd.read_parquet(os.path.join(root_path, 'trade_test.parquet'))

In [ ]:
train_data = pd.read_csv(os.path.join(root_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(root_path, 'test.csv'))

In [ ]:
def cal_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def cal_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Remember that log(x / y) = log(x) - log(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

In [ ]:
# Function to get group stats for different windows (seconds in bucket)
def get_stats_window(df, seconds_in_bucket, create_feature_dict, add_suffix = False):
    # Group by the window
    df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
    # Rename columns joining suffix
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    # Add a suffix to differentiate windows
    if add_suffix:
        df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
    return df_feature

def book_preprocessor(book_df, stock_id):
    book_df['wap1'] = cal_wap1(book_df)
    book_df['wap2'] = cal_wap2(book_df)
    book_df['wap3'] = calc_wap3(book_df)
    book_df['wap4'] = calc_wap4(book_df)
    # Calculate log returns
    book_df['log_return1'] = book_df.groupby(['time_id'])['wap1'].apply(log_return)
    book_df['log_return2'] = book_df.groupby(['time_id'])['wap2'].apply(log_return)
    
    # Calculate wap balance
    book_df['wap_balance'] = abs(book_df['wap1'] - book_df['wap2'])
    # Calculate spread
    book_df['price_spread'] = (book_df['ask_price1'] - book_df['bid_price1']) / ((book_df['ask_price1'] + book_df['bid_price1']) / 2)
    book_df['price_spread2'] = (book_df['ask_price2'] - book_df['bid_price2']) / ((book_df['ask_price2'] + book_df['bid_price2']) / 2)
    book_df['bid_spread'] = book_df['bid_price1'] - book_df['bid_price2']
    book_df['ask_spread'] = book_df['ask_price1'] - book_df['ask_price2']
    book_df["bid_ask_spread"] = abs(book_df['bid_spread'] - book_df['ask_spread'])
    book_df['total_volume'] = (book_df['ask_size1'] + book_df['ask_size2']) + (book_df['bid_size1'] + book_df['bid_size2'])
    book_df['volume_imbalance'] = abs((book_df['ask_size1'] + book_df['ask_size2']) - (book_df['bid_size1'] + book_df['bid_size2']))
    
    book_df['deep_unbalance1'] = (book_df['bid_size1'] - book_df['ask_size1']) / (book_df['bid_size1'] + book_df['ask_size1'])
    book_df['deep_unbalance2'] = (book_df['bid_size2'] - book_df['ask_size2']) / (book_df['bid_size2'] + book_df['ask_size2'])
    
    book_df['volume_pressure'] = book_df['bid_size1'] + book_df['bid_size2'] - book_df['ask_size1'] - book_df['ask_size2']
    book_df['width_pressure'] = (book_df['bid_price1'] - book_df['bid_price2'] - book_df['ask_price2'] + book_df['ask_price1']) / (book_df['bid_price1'] - book_df['bid_price2'] + book_df['ask_price2'] - book_df['ask_price1'])
    book_df['mid_price'] = (book_df['ask_price1'] + book_df['bid_price1']) / 2
#     book_df['buy_press_fct1'] = book_df['mid_price'] / (book_df['mid_price'] - book_df['bid_price1'])
#     book_df['buy_press_fct2'] = book_df['mid_price'] / (book_df['mid_price'] - book_df['bid_price2'])
#     book_df['sell_press_fct1'] = book_df['mid_price'] / (book_df['ask_price1'] - book_df['mid_price'])
#     book_df['sell_press_fct2'] = book_df['mid_price'] / (book_df['ask_price2'] - book_df['mid_price'])
#     book_df['buy_press_sum'] = book_df['buy_press_fct1'] + book_df['buy_press_fct2']
#     book_df['sell_press_sum'] = book_df['sell_press_fct1'] + book_df['sell_press_fct2']
#     book_df['buy_press'] = (book_df['bid_size1'] * book_df['buy_press_fct1'] + book_df['bid_size2'] * book_df['buy_press_fct2']) / book_df['buy_press_sum']    
#     book_df['sell_press'] = (book_df['ask_size1'] * book_df['sell_press_fct1'] + book_df['ask_size2'] * book_df['sell_press_fct2']) / book_df['sell_press_sum']
#     book_df['order_unbalance'] = np.log(book_df['buy_press']) - np.log(book_df['sell_press'])
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum,np.std,np.mean],
        'wap2': [np.sum,np.std,np.mean],
        'log_return1': [realized_volatility,np.std,np.mean],
        'log_return2': [realized_volatility,np.std,np.mean],
        'wap_balance': [np.sum,np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        'bid_ask_spread':[np.sum, np.mean, np.std],
        'volume_pressure': [np.sum, np.mean, np.std],
        'deep_unbalance1': [np.sum, np.mean, np.std],
        'deep_unbalance2': [np.sum, np.mean, np.std],
        'width_pressure' :[np.sum, np.mean, np.std],
        'mid_price' : [np.mean, np.std]
#         'buy_press_fct1' : [np.mean, np.std],
#         'buy_press_fct2' : [np.mean, np.std],
#         'sell_press_fct1': [np.mean, np.std],
#         'sell_press_fct2': [np.mean, np.std],
#         'buy_press_sum':[np.mean, np.sum, np.std],
#         'sell_press_sum':[np.mean, np.sum, np.std],
#         'buy_press' : [np.mean, np.sum, np.std],
#         'sell_press' : [np.mean, np.sum, np.std],
#         'order_unbalance':[np.mean, np.sum, np.std]
    }
    
    # Get the stats for different windows
    df_feature = get_stats_window(df=book_df, create_feature_dict=create_feature_dict, seconds_in_bucket=0, add_suffix=False)
    df_feature_500 = get_stats_window(df=book_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 500, add_suffix = True)
#     df_feature_400 = get_stats_window(df=book_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(df=book_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 300, add_suffix = True)
#     df_feature_200 = get_stats_window(df=book_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(df=book_df, create_feature_dict=create_feature_dict,seconds_in_bucket = 100, add_suffix = True)
    
    
     # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
#     df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
#     df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500', 'time_id__300','time_id__100'], axis = 1, inplace = True)
    # Create row_id so we can merge
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [ ]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(trade_df, stock_id):
    trade_df['log_return'] = trade_df.groupby('time_id')['price'].apply(log_return)
    trade_df['amount']= trade_df['price'] * trade_df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max,np.std,np.min],
         'amount':[np.sum,np.max,np.min]
    }

    # Get the stats for different windows
    df_feature = get_stats_window(df=trade_df, create_feature_dict=create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(df=trade_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 500, add_suffix = True)
#     df_feature_400 = get_stats_window(df=trade_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(df=trade_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 300, add_suffix = True)
#     df_feature_200 = get_stats_window(df=trade_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(df=trade_df, create_feature_dict=create_feature_dict, seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(trade_stock):
        # 用作 groupby 里的函数，用每笔成交价格的变化率乘以该笔成交价格的成交量，然后求和
        price_rtn = (trade_stock['price'].diff() / trade_stock['price']) * 100
        val = price_rtn * trade_stock['size']
        power = val.sum()
        return power
    
    def cal_mean(trade_stock):
        return np.mean(trade_stock['price'].values)
    
    def cal_std(trade_stock):
        return np.std(trade_stock['price'].values)
    
    def cal_singularity_price(trade_stock):
        mean = np.mean(trade_stock['price'].values)
        std = np.std(trade_stock['price'].values)
        upper = mean + std
        lower = mean - std
        return np.sum((trade_stock['price'] >= upper) | (trade_stock['price'] <= lower))
    
    def cal_mean_vol(trade_stock):
        return np.mean(trade_stock['size'].values)
    
    def cal_std_vol(trade_stock):
        return np.std(trade_stock['size'].values)
    
    def cal_singularity_vol(trade_stock):
        mean = np.mean(trade_stock['size'].values)
        std = np.std(trade_stock['size'].values)
        upper = mean + std
        lower = mean - std
        return np.sum((trade_stock['size'] >= upper) | (trade_stock['size'] <= lower))
    
    def cal_order_count_width(trade_stock):
        return trade_stock['order_count'].max() - trade_stock['order_count'].min()
    
    def cal_f_max(trade_stock):
        # 某只票在某个时间点的所有成交价格，大于平均值的数量
        return np.sum(trade_stock['price'].values > np.mean(trade_stock['price'].values))
    
    def cal_f_min(trade_stock):
        # 某只票在某个时间点的所有成交价格，小于平均值的数量
        return np.sum(trade_stock['price'].values < np.mean(trade_stock['price'].values))
    
    def cal_max(trade_stock):
        # 计算价格变化量大于0的数量
        return np.sum(np.diff(trade_stock['price'].values) > 0)
    
    def cal_min(trade_stock):
        # 计算价格变化量小于0的数量
        return np.sum(np.diff(trade_stock['price'].values) < 0)
    
    def cal_abs_diff(trade_stock):
        # 计算价格距离平均值的距离，并求距离的中位数
        return np.median(np.abs(trade_stock['price'].values - np.mean(trade_stock['price'].values)))
    
    def cal_energy(trade_stock):
        # 计算价格的平方的平均值
        return np.mean(trade_stock['price'].values ** 2)
    
    def cal_iqr_p(trade_stock):
        # 计算价格的75分位与25分位之差
        return np.percentile(trade_stock['price'].values, 75) - np.percentile(trade_stock['price'].values, 25)
    
    def cal_abs_diff_v(trade_stock):
        # 计算成交量离平均值的距离，并求距离的中位数
        return np.median(np.abs(trade_stock['size'].values - np.mean(trade_stock['size'].values)))  
    
    def cal_energy_v(trade_stock):
        # 计算成交量的平方和
        return np.sum(trade_stock['size'].values**2)
    
    def iqr_p_v(trade_stock):
        return np.percentile(trade_stock['size'].values,75) - np.percentile(trade_stock['size'].values,25)
    
    def price_volume_corr(trade_stock):
        return trade_stock['price'].corr(trade_stock['size'])
    
    def price_volume_corr2(trade_stock):
        diff1 = trade_stock['price'].diff()
        diff2 = trade_stock['size'].diff()
        return diff1.corr(diff2)
    
    def weighted_size(trade_stock):
        return (trade_stock['size'] * trade_stock['order_count'] / trade_stock['order_count'].sum()).sum()
    
    def weighted_price(trade_stock):
        return (trade_stock['price'] * trade_stock['order_count'] / trade_stock['order_count'].sum()).sum()
    
    
    
    
    df_lr = pd.DataFrame()
    df_lr_group = trade_df.groupby('time_id')
    df_lr['tendency'] = df_lr_group.apply(tendency)
    df_lr['mean'] = df_lr_group.apply(cal_mean)
    df_lr['std'] = df_lr_group.apply(cal_std)
    df_lr['price_singularity'] = df_lr_group.apply(cal_singularity_price)
    df_lr['vol_mean'] = df_lr_group.apply(cal_mean_vol)
    df_lr['vol_std'] = df_lr_group.apply(cal_std_vol)
    df_lr['vol_singularity'] = df_lr_group.apply(cal_singularity_vol)
    df_lr['order_count_width'] = df_lr_group.apply(cal_order_count_width)
    df_lr['f_max'] = df_lr_group.apply(cal_f_max)
    df_lr['f_min'] = df_lr_group.apply(cal_f_min)
    df_lr['df_max'] = df_lr_group.apply(cal_max)
    df_lr['df_min'] = df_lr_group.apply(cal_min)
    df_lr['abs_diff'] = df_lr_group.apply(cal_abs_diff)
    df_lr['energy'] = df_lr_group.apply(cal_energy)
    df_lr['iqr_p'] = df_lr_group.apply(cal_iqr_p)
    df_lr['abs_diff_v'] = df_lr_group.apply(cal_abs_diff_v)
    df_lr['energy_v'] = df_lr_group.apply(cal_energy_v)
    df_lr['iqr_p_v'] = df_lr_group.apply(iqr_p_v)
    df_lr['pv_corr'] = df_lr_group.apply(price_volume_corr)
    df_lr['pv_corr2'] = df_lr_group.apply(price_volume_corr2)
    df_lr['weighted_size'] = df_lr_group.apply(weighted_size)
    df_lr['weighted_price'] = df_lr_group.apply(weighted_price)
    df_lr.reset_index(inplace=True)
    # Merge all
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
#     df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
#     df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500', 'time_id__300','time_id','time_id__100'], axis = 1, inplace = True)
    df_feature = df_feature.add_prefix('trade_')
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

In [ ]:
def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 
                'log_return1_realized_volatility_500', 'log_return2_realized_volatility_500', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_500',
                'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_100']


    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

In [ ]:
# Parrallel for loop
def preprocesor_help(stock_id, is_train, cache):
    # Train
    t0 = dt.datetime.now()
    if is_train:
        file_path_book = root_path + "/book_train.parquet/stock_id=" + str(stock_id)
        file_path_trade = root_path + "/trade_train.parquet/stock_id=" + str(stock_id)
    # Test
    else:
        file_path_book = root_path + "/book_test.parquet/stock_id=" + str(stock_id)
        file_path_trade = root_path + "/trade_test.parquet/stock_id=" + str(stock_id)

    # Preprocess book and trade data and merge them
    book_df = pd.read_parquet(file_path_book)
    trade_df = pd.read_parquet(file_path_trade)
    book_stock_proprocess = book_preprocessor(book_df, str(stock_id))
    trade_stock_proprocess = trade_preprocessor(trade_df, str(stock_id))
    df_tmp = pd.merge(book_stock_proprocess, trade_stock_proprocess, on = 'row_id', how = 'left')
    t1 = dt.datetime.now()
    print("stock_id: {} finishes, total time: {}".format(stock_id, ((t1-t0).total_seconds())))
    cache.append(df_tmp)


# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    # Use parallel api to call paralle for loop
#     df = Parallel(n_jobs = -1, verbose = 1)(delayed(preprocesor_help)(stock_id) for stock_id in list_stock_ids)
    pool = Pool(4)
    cache = multiprocessing.Manager().list()
#     cache = []
    for stock_id in list_stock_ids:
        pool.apply_async(preprocesor_help, args=(stock_id, is_train, cache, ))
#         preprocesor_help(stock_id, True, cache)
    pool.close()
    pool.join()
    cache_list = list(cache)
#     cache_list = cache
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(cache_list, ignore_index = True)
    return df

In [ ]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

In [ ]:
# Read train and test
train, test = read_train_test()

In [ ]:
# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()

# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)

In [ ]:
train = train.merge(train_, on = ['row_id'], how = 'left')
test = test.merge(test_, on = ['row_id'], how = 'left')

In [ ]:
gc.collect()

In [ ]:
train = get_time_stock(train)
test = get_time_stock(test)
del train_
del test_

In [ ]:
# replace by order sum (tau)
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_500'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_500'] )
train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_100'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_100'] )

In [ ]:
train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

# delta tau
train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

In [ ]:
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from numpy.random import seed
seed(114)

In [ ]:
# 将train里的特征用quantile 进行变化
colNames = list(train)
colNames.remove('time_id')
colNames.remove('target')
colNames.remove('row_id')
colNames.remove('stock_id')

train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)

In [ ]:
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
train_p.replace([-np.inf, np.inf], np.nan, inplace=True)
train_p.iloc[:, [35,70,88, 66, 12]] = train_p.iloc[:, [35,70,88, 66, 12]].ffill()
train_p.dropna(axis=1, inplace=True, how='any')
corr = train_p.corr()
ids = corr.index
kmeans = KMeans(n_clusters=7, random_state=0).fit(train_p.T.values)

In [ ]:
cat_labels = kmeans.labels_

In [ ]:
l = []
for n in range(7):  # 获取每一类有哪几个index
    l.append (ids[cat_labels == n].values.tolist())

In [ ]:
mat = []
matTest = []
n = 0
for ind in l:
    # 把kmeans 分类出来的结果都存下来，然后按time_id 取feature的平均值，最后的结果是在同一时间，这一类所有股票的
    # feature 平均值
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind)]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf ) 
    n+=1

In [ ]:
stk_concat_train = pd.concat(mat).reset_index()
stk_concat_train.drop(columns=['target'],inplace=True)
stk_concat_test = pd.concat(matTest).reset_index()

In [ ]:
stk_concat_test = pd.concat([stk_concat_test, stk_concat_train.loc[stk_concat_train.time_id == 5]])

In [ ]:
# 把所有的feature 的名称加上后缀，后缀为kmeans 分出来的类
stk_pivot_train = stk_concat_train.pivot(index='time_id', columns='stock_id')
stk_pivot_train.columns = ["_".join(x) for x in stk_pivot_train.columns.ravel()]
stk_pivot_train.reset_index(inplace=True)

In [ ]:
# 把所有的feature 的名称加上后缀，后缀为kmeans 分出来的类
stk_pivot_test = stk_concat_test.pivot(index='time_id', columns='stock_id')
stk_pivot_test.columns = ["_".join(x) for x in stk_pivot_test.columns.ravel()]
stk_pivot_test.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1',
     'deep_unbalance1_mean_0c1',
     'deep_unbalance1_mean_1c1',
     'deep_unbalance1_mean_2c1',
     'deep_unbalance1_mean_3c1',
     'deep_unbalance1_mean_4c1',
     'deep_unbalance1_mean_6c1']
# 把用kmeans 分类出来的信息按照 time_id, merge到原train
train_merge = pd.merge(train, stk_pivot_train[nnn], how='left',on='time_id')
test_merge = pd.merge(test, stk_pivot_test[nnn], how='left',on='time_id')
del book_train
del book_test
del trade_train
del trade_test
del train_p
del stk_pivot_train
del stk_pivot_test
del stk_concat_train
del stk_concat_test
del train
del test
del mat
del matTest
del newDf

In [ ]:
import tensorflow as tf
from sklearn import model_selection
tf.random.set_seed(114)
from tensorflow import keras
import numpy as np
from keras import backend as K
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
import lightgbm as lgb
import warnings

In [ ]:
def model(train, useful_features, categoricals, target, n_splits, num_leaves, max_depth, min_child_weight, feature_fraction, lambda_l1, lambda_l2, 
          bagging_fraction, min_data_in_leaf, learning_rate, n_estimators):
     
    params =  {'num_leaves': int(num_leaves),  
        'max_depth' : int(max_depth),
       'min_child_weight': min_child_weight,
       'feature_fraction': feature_fraction,
       'bagging_fraction': bagging_fraction,
       'min_data_in_leaf': int(min_data_in_leaf), 
       'objective': 'regression',
       "metric": 'rmse',
       'learning_rate': learning_rate, 
       "boosting_type": "gbdt",
       "bagging_seed": 11,
       "verbosity": -1,
       'random_state': 46,
       'lambda_l1': lambda_l1,  
       'lambda_l2': lambda_l2, 
       'n_estimators': int(n_estimators),
       'early_stopping': 150,
       'n_jobs': -1       
}

    def run_lgb(reduce_train, useful_features, categoricals, target, n_splits = n_splits):
        #useful_features.remove('installation_id')
        rmse_score_list = []
        kf = model_selection.KFold(n_splits = n_splits, random_state = 42, shuffle = True)
        oof_predict = np.zeros((len(reduce_train), ))
        for fold, (train_index, test_index) in enumerate(kf.split(reduce_train)):
            X_train = reduce_train[useful_features].iloc[train_index]
            X_val = reduce_train[useful_features].iloc[test_index]
            y_train = reduce_train[target].iloc[train_index]
            y_val = reduce_train[target].iloc[test_index]

            train_weights = 1 / np.square(y_train)
            val_weights = 1 / np.square(y_val)
            train_dataset = lgb.Dataset(X_train, y_train, weight = train_weights, categorical_feature = [categoricals])
            val_dataset = lgb.Dataset(X_val, y_val, weight = val_weights, categorical_feature = [categoricals])

            lgb_model = lgb.train(params, train_dataset, num_boost_round = params['n_estimators'], 
                                  valid_sets = [train_dataset, val_dataset],
                                 early_stopping_rounds = params['early_stopping'],
                                 feval = feval_rmspe, verbose_eval=False)

            val_predict = lgb_model.predict(X_val)
             # Add predictions to the out of folds array
            oof_predict[test_index] = val_predict

        rmspe_score = rmspe(reduce_train['target'], oof_predict)
        return -rmspe_score

    return run_lgb(train, useful_features, categoricals, target)

In [ ]:
useful_features = list(train_merge)
useful_features.remove('row_id')
useful_features.remove('target')
useful_features.remove('time_id')
target = 'target'
bounds_LGB = {
    'num_leaves' : (50, 100),
    'max_depth': (2, 80),
    'min_child_weight' : (0.01, 0.6),
    'min_data_in_leaf' : (80, 1000),
    'feature_fraction' : (0.1, 0.95),
    'lambda_l1': (0, 10),
    'lambda_l2': (0, 10),
    'bagging_fraction': (0.2, 1),
    'learning_rate': (0.01, 0.8),
    'n_estimators' : (500,7000)
}

In [ ]:
warnings.filterwarnings("ignore")
from functools import partial

In [ ]:
# 对每一组训练一次模型，用lightgbm 和 Bayesian 优化
init_points = 12
n_iter = 12 
best_param = {}
for idx, indiv in enumerate(l):
    kmeans_stock = train_merge.loc[train_merge['stock_id'].isin(indiv)]
    kmeans_stock.reset_index(inplace=True, drop=True)
    partial_model = partial(model, kmeans_stock, useful_features, categoricals='stock_id',target=target, n_splits = 5)
    LGB_BO = BayesianOptimization(partial_model, bounds_LGB, random_state = 1029)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')   
        LGB_BO.maximize(init_points = init_points, n_iter = n_iter, acq='ei', alpha=1e-6)
    best_param[idx] = LGB_BO.max['params']

In [ ]:
def param_transform(params):
    params_new =  {'num_leaves': int(params['num_leaves']),  
            'max_depth' : int(params['max_depth']),
           'min_child_weight': params['min_child_weight'],
           'feature_fraction': params['feature_fraction'],
           'bagging_fraction': params['bagging_fraction'] ,
           'min_data_in_leaf': int(params['min_data_in_leaf']), 
           'objective': 'regression',
           "metric": 'rmse',   
           'learning_rate': params['learning_rate'], 
           "boosting_type": "gbdt",
           "bagging_seed": 11,
           "verbosity": -1,
           'random_state': 46,
           'lambda_l1': params['lambda_l1'],  
           'lambda_l2': params['lambda_l2'], 
           'n_estimators': int(params['n_estimators']),
           'early_stopping': 150,
           'n_jobs': -1       
    }
    return params_new

In [ ]:
def train_with_lgb(train_data, test_data, params):
    model_name = 'NN'
    n_folds = 5
    counter = 1
    target_name = 'target'
    validation_scores = []
    test_result = {}
    oof_predictions = np.zeros(train_data.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(test_data.shape[0])
    
    kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2020)
    features_to_consider = list(train_data)
    features_to_consider.remove('time_id')
    features_to_consider.remove('target')
    features_to_consider.remove('row_id')
    num_features = features_to_consider.copy()
    num_features.remove('stock_id')
    train_data[num_features] = train_data[num_features].fillna(train_data[num_features].mean())
    test_data[num_features] = test_data[num_features].fillna(test_data[num_features].mean())
    
    
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
        X_train = train_data.loc[train_idx, features_to_consider]
        y_train = train_data.loc[train_idx, target_name]
        X_test = train_data.loc[val_idx, features_to_consider]
        y_test = train_data.loc[val_idx, target_name]
        
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_test)
        train_dataset = lgb.Dataset(X_train, y_train, weight = train_weights)
        val_dataset = lgb.Dataset(X_test, y_test, weight = val_weights)
        
        model = lgb.train(params = params,
                          num_boost_round = 1300,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=50,
                          feval = feval_rmspe)
        
         # Add predictions to the out of folds array
        oof_predictions[val_idx] = model.predict(X_test)
        # Predict the test set
        test_predictions += model.predict(test_data[features_to_consider]) / 5
#         break
    rmspe_score = rmspe(train_data['target'], oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')  
    lgb.plot_importance(model, max_num_features=30)
    return test_predictions

test_predict_list = []
train_predict_list = []


for idx, indiv in enumerate(l):
    kmeans_stock = train_merge.loc[train_merge['stock_id'].isin(indiv)]
    kmeans_stock_test = test_merge.loc[test_merge['stock_id'].isin(indiv)]
    train_predictions = np.zeros(kmeans_stock.shape[0])
    test_predictions = np.zeros(kmeans_stock_test.shape[0])
    kf = model_selection.KFold(n_splits = 5, random_state = 42, shuffle = True)
    params = best_param[idx]
    params = param_transform(params)
    kmeans_stock_test_valid = kmeans_stock_test[useful_features]
    for fold, (train_index, test_index) in enumerate(kf.split(kmeans_stock)):
        X_train = kmeans_stock[useful_features].iloc[train_index]
        X_val = kmeans_stock[useful_features].iloc[test_index]
        y_train = kmeans_stock[target].iloc[train_index]
        y_val = kmeans_stock[target].iloc[test_index]

        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(X_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(X_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        lgb_model = lgb.train(params, train_dataset, num_boost_round = params['n_estimators'], 
                              valid_sets = [train_dataset, val_dataset],
                             early_stopping_rounds = params['early_stopping'],
                             feval = feval_rmspe, verbose_eval=False)
        
        try:
            train_predictions[test_index] += lgb_model.predict(X_val)
            test_predictions += lgb_model.predict(kmeans_stock_test_valid) / 5
        except:
            test_predictions = np.zeros(kmeans_stock_test.shape[0])
    kmeans_stock['target_pred'] = train_predictions
    kmeans_stock_test['target'] = test_predictions
    train_predict_list.append(kmeans_stock)
    test_predict_list.append(kmeans_stock_test)
    
test_submit = pd.concat(test_predict_list)
train_pred = pd.concat(train_predict_list)
train_pred.sort_values('row_id', inplace=True)
test_submit.sort_values('row_id', inplace=True)

del test_predict_list
del kmeans_stock
del kmeans_stock_test
del test_predictions
gc.collect()

In [ ]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': swish})

In [ ]:
def root_mean_squared_per_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [ ]:
for col in colNames:
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
    train_merge[col] = qt.fit_transform(train_merge[[col]])
    test_merge[col] = qt.transform(test_merge[[col]])   

In [ ]:
hidden_units = (252, 128, 64, 32)
stock_embedding_size = 32
cat_data = train_merge['stock_id']

In [ ]:
num_data_dim = train_merge.shape[1] - 4

In [ ]:
def base_model():
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')  # stock id input, 维数是1维
    num_input = keras.Input(shape=(num_data_dim,), name='num_data')  # num_data input, 维数是 266维


    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)  #  降维，将（none, 24, 1）,降成 （24,1）
    out = keras.layers.Concatenate()([stock_flattened, num_input]) # shape (None, 386), 其中num_input shape 为 (362， )
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        out = keras.layers.Dropout(rate=0.02,seed=111)(out)
        

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model
def train_with_nn(train_data, test_data):
    model_name = 'NN'
    n_folds = 5
    counter = 1
    target_name = 'target'
    validation_scores = []
    oob_predictions = np.zeros(train_data.shape[0])
    test_result = {}
    kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2020)
    features_to_consider = list(train_data)
    features_to_consider.remove('time_id')
    features_to_consider.remove('target')
    features_to_consider.remove('row_id')
    num_features = features_to_consider.copy()
    num_features.remove('stock_id')
    train_data[num_features] = train_data[num_features].fillna(train_data[num_features].mean())
    test_data[num_features] = test_data[num_features].fillna(test_data[num_features].mean())
    test_data['target'] = 0
    scaler = MinMaxScaler(feature_range=(-1, 1))  
    
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
        X_train = train_data.loc[train_idx, features_to_consider]
        y_train = train_data.loc[train_idx, target_name]
        X_test = train_data.loc[val_idx, features_to_consider]
        y_test = train_data.loc[val_idx, target_name]
        model = base_model()
        model.compile(
            keras.optimizers.Adam(learning_rate=0.0061),
            loss=root_mean_squared_per_error
        )
        
        num_data_train = X_train[num_features]
        num_data_val = X_test[num_features]       
        num_data_train = scaler.fit_transform(num_data_train.values) 
        num_data_val = scaler.transform(num_data_val.values)
        cat_data_train = X_train['stock_id']
        cat_data_val = X_test['stock_id']
        
        model.fit([cat_data_train, num_data_train], 
              y_train,               
              batch_size=2048,
              epochs=200,
              validation_data=([cat_data_val, num_data_val], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
              verbose = 1)

        preds = model.predict([cat_data_val, num_data_val]).reshape(1,-1)[0]
        oob_predictions[val_idx] = preds
        score = round(rmspe(y_true = y_test, y_pred = preds), 5)
        print('Fold {} {}: {}'.format(counter, model_name, score))
        validation_scores.append(score)
        
        test_data_transformed = scaler.transform(test_data[num_features].values)
        test_score = model.predict([test_data['stock_id'], test_data_transformed]).reshape(1,-1)[0].clip(0,1e10)
        print("current test score is: {}".format(test_score))
        test_data['target'] += (test_score / n_folds)
        del test_score
#         test_result[fold] = (test_score)
#         break
    return oob_predictions, test_data

In [ ]:
oob_predictions, test_data = train_with_nn(train_merge, test_merge)

In [ ]:
train_merge['pred'] = oob_predictions

In [ ]:
train_merge.sort_values('row_id', inplace=True)
test_merge.sort_values('row_id', inplace=True)
lambda_bound = {'lambda1': (0,1)}
def result_optimizer(lambda1):
    result = lambda1 * train_merge['pred'].values + (1 - lambda1) * train_pred['target_pred'].values
    return -round(rmspe(y_true = train_pred['target'].values, y_pred = result), 5)

In [ ]:
lambda1 = 0.1702
# pred_lgb_m = predictions_lgb * 0.615 + predictions_lgb2 * 0.385   
test_data['target'] = test_data['target'] * lambda1 + test_submit['target'] * (1-lambda1) 
test_data[['row_id', 'target']].to_csv('submission.csv',index = False)